In [ ]:
import vaex
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from glob import glob

In [ ]:
files = glob(<data files path>)

In [ ]:
tmass_files = glob(<2MASS files path>)

In [ ]:
data = vaex.open_many(files)
data = data.filter(data['Jmag'] < 11)
data

In [ ]:
tmass = vaex.open_many(tmass_files)
tmass

In [ ]:
# check the plot, optional
data.viz.scatter("Jmag - Kmag", "Jmag", s=1,length_check=False)
plt.gca().invert_yaxis()

In [ ]:
tmass.viz.scatter("Jmag - Kmag", "Jmag", s=1,length_check=False)
plt.gca().invert_yaxis()

In [ ]:
# partition in J and J-K
dJ = np.linspace(5, 11, 51)
dJK = np.linspace(0, 1, 31)

In [ ]:
# display histogram
dataH, _, _ = np.histogram2d((data["Jmag"]-data["Kmag"]).to_numpy(), data["Jmag"].to_numpy(), bins=(dJK, dJ))
dataH = dataH.T

fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(131, title='Data')
plt.imshow(dataH, interpolation='nearest', origin='lower',
        extent=[dJK[0], dJK[-1], dJ[0], dJ[-1]], aspect='auto')
plt.show()

In [ ]:
tmassH, _, _ = np.histogram2d((tmass["Jmag"]-tmass["Kmag"]).to_numpy(), tmass["Jmag"].to_numpy(), bins=(dJK, dJ))
tmassH = tmassH.T

fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(131, title='Data')
plt.imshow(tmassH, interpolation='nearest', origin='lower',
        extent=[dJK[0], dJK[-1], dJ[0], dJ[-1]], aspect='auto')
plt.show()

In [ ]:
# calculate completeness
comp = dataH/tmassH
comp = np.nan_to_num(comp)

In [ ]:
# display it
fig = plt.figure(figsize=(25, 5))
ax = fig.add_subplot(131, title='Completeness')
plt.imshow(comp, interpolation='nearest', origin='lower',
        extent=[dJK[0], dJK[-1], dJ[0], dJ[-1]], aspect='auto')
plt.colorbar()
# ax.invert_yaxis()
plt.show()

In [ ]:
# convert into dataframe

df = pd.DataFrame(comp, columns=(dJK[:-1]+dJK[1:])/2)
df.insert(0, "J", (dJ[:-1]+dJ[1:])/2)
df

In [ ]:
# save as hdf5
vaex.from_pandas(df).export("comp.hdf5", progress=True)